In [3]:
#Reference: https://gist.github.com/PawaritL/ec7136c0b718ca65db6df1c33fd1bb11, #dggal

from shapely.ops import transform

# -----------------------------------------------------------
# Detect whether a geometry crosses the antimeridian
# -----------------------------------------------------------
def check_crossing(lon1: float, lon2: float, validate: bool = True):
    """
    Assuming a minimum travel distance between two provided longitude coordinates,
    checks if the 180th meridian (antimeridian) is crossed.
    """
    if validate and any(abs(x) > 180.0 for x in [lon1, lon2]):
        raise ValueError("longitudes must be in degrees [-180.0, 180.0]")
    return abs(lon2 - lon1) > 180.0


def check_crossing_geom(geom):
    """
    Check if a geometry crosses the antimeridian (180th meridian).

    Args:
        geom: Shapely geometry (Polygon, MultiPolygon)

    Returns:
        bool: True if any part of the geometry crosses the antimeridian, False otherwise
    """
    crossed = False
    try:
        # Handle None or empty geometries
        if geom is None or geom.is_empty:
            return False

        # Handle multi-geometries
        if hasattr(geom, "geoms"):
            # MultiPolygon
            for sub_geom in geom.geoms:
                if check_crossing_geom(sub_geom):
                    crossed = True
                    break
            return crossed

        # Handle single Polygon
        if geom.geom_type == "Polygon":
            # Check if exterior has coordinates
            if geom.exterior is None or len(geom.exterior.coords) == 0:
                return False

            # Check exterior ring only
            p_init = geom.exterior.coords[0]
            for p in range(1, len(geom.exterior.coords)):
                px = geom.exterior.coords[p]
                try:
                    if check_crossing(p_init[0], px[0]):
                        crossed = True
                        break
                except ValueError:
                    crossed = True
                    break
                p_init = px
    except Exception:
        return False

    return crossed


# -----------------------------------------------------------
# Shift to WEST side (only when crossing)
# -----------------------------------------------------------
def westshift(geom):
    if not check_crossing_geom(geom):
        return geom  # do nothing

    def shift(x, y):
        if x > 0:      # 170 → -190
            x -= 360      
        return (x, y)

    return transform(shift, geom)


# -----------------------------------------------------------
# Shift to EAST side (only when crossing)
# -----------------------------------------------------------
def eastshift(geom):
    if not check_crossing_geom(geom):
        return geom  # do nothing

    def shift(x, y):
        if x < 0:      # -170 → 190
            x += 360       
        return (x, y)

    return transform(shift, geom)


# -----------------------------------------------------------
# Balanced shift based on centroid location
# -----------------------------------------------------------
def balancedshift(geom):
    """
    Shift geometry based on centroid location:
    - If centroid longitude < -180: apply west shift
    - If centroid longitude >= 180: apply east shift
    - Otherwise: determine based on centroid position
    
    Args:
        geom: Shapely geometry (Polygon, MultiPolygon)
    
    Returns:
        Shapely geometry: shifted geometry
    """
    if not check_crossing_geom(geom):
        return geom  # do nothing if not crossing
    
    # Get centroid longitude
    centroid = geom.centroid
    centroid_lon = centroid.x
    
    # Apply shift based on centroid longitude
    if centroid_lon < -180:
        return westshift(geom)
    elif centroid_lon >= 180:
        return eastshift(geom)
    else:
        # For geometries with centroid in [-180, 180], shift based on sign
        # Negative (western hemisphere) -> west shift, positive -> east shift
        if centroid_lon < 0:
            return westshift(geom)
        else:
            return eastshift(geom)


In [13]:
import geopandas as gpd
from shapely import wkt

# Get the test polygon
poly = wkt.loads("POLYGON((150 40, 150 50, -160 50, -160 40, 150 40))")

# Apply shifts
west = westshift(poly)
east = eastshift(poly)
balanced = balancedshift(poly)

# Create GeoDataFrame with all geometries
gdf = gpd.GeoDataFrame({
    'name': ['original', 'west', 'east', 'balanced'],
    'geometry': [poly, west, east, balanced]
}, crs='EPSG:4326')

print("GeoDataFrame created with all shifts:")
print(gdf)
print()

# Save to a single GeoJSON file
output_file = "all_shifts.geojson"
gdf.to_file(output_file, driver='GeoJSON')
print(f"Saved all shifts to: {output_file}")


GeoDataFrame created with all shifts:
       name                                           geometry
0  original  POLYGON ((150 40, 150 50, -160 50, -160 40, 15...
1      west  POLYGON ((-210 40, -210 50, -160 50, -160 40, ...
2      east  POLYGON ((150 40, 150 50, 200 50, 200 40, 150 ...
3  balanced  POLYGON ((-210 40, -210 50, -160 50, -160 40, ...

Saved all shifts to: all_shifts.geojson


In [1]:
import geopandas as gpd
from shapely import wkt

# Your WKT polygon
polygon_wkt = "POLYGON((170 40, 170 50, -170 50, -170 40, 170 40))"

# Create a GeoSeries
gdf = gpd.GeoSeries([wkt.loads(polygon_wkt)], crs="EPSG:4326")

# Area in degrees² (approx, just multiply lon/lat differences)
area_deg2 = gdf.area[0]
print("Area in degree²:", area_deg2)

# Optional: Project to an equal-area projection to get km²
gdf_proj = gdf.to_crs("EPSG:6933")  # World Cylindrical Equal Area
area_km2 = gdf_proj.area[0] / 1e6   # convert m² to km²
print("Approximate area in km²:", area_km2)

Area in degree²: 3400.0
Approximate area in km²: 29753321.48249786


C:\Users\TECHCRAFT-QDThang\AppData\Local\Temp\ipykernel_32092\2890344483.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  area_deg2 = gdf.area[0]
